Import MNIST Data

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import OneHotEncoder

In [2]:
image, y_train = tfds.as_numpy(tfds.load(
    'mnist',
    split='train',
    batch_size=-1,
    as_supervised=True,
))

x_train=image.reshape(60000,-1)
print(type(y_train), y_train.shape)
enc = OneHotEncoder()
y_train=y_train.reshape(-1, 1)
enc.fit(y_train)
y_train=enc.transform(y_train).toarray()
print(type(y_train), y_train.shape)
print(y_train[:10])

<class 'numpy.ndarray'> (60000,)
<class 'numpy.ndarray'> (60000, 10)
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [3]:
image, y_test = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

x_test=image.reshape(10000,-1)
enc = OneHotEncoder()
y_test=y_test.reshape(-1, 1)
enc.fit(y_test)
y_test=enc.transform(y_test).toarray()

Define the training data placeholders

In [4]:
# Python optimization variables
learning_rate = 0.5
epochs = 10
batch_size = 100

# Declare the training data placeholders
tf.compat.v1.disable_eager_execution()
# Input x - for 28 x 28 pixels = 784
x = tf.compat.v1.placeholder(tf.float32, shape=[None, 784], name="X")
# Now declare the output data placeholder - 10 digits
y = tf.compat.v1.placeholder(tf.float32, shape=[None, 10], name="Y")

Define the weights and the biases

In [5]:
# Now declare the weights connecting the input to the hidden layer (300 neurons)
W1 = tf.Variable(name="Weight1", dtype=tf.float32, shape=[784, 300], initial_value=tf.random.normal(mean=0.0, stddev=0.01, shape=[784, 300], seed=42))
b1 = tf.Variable(name="Bias1", dtype=tf.float32, initial_value=tf.constant(0., shape=[300], dtype=tf.float32))
# And the weights connecting the hidden layer to the output layer
W2 = tf.Variable(name="Weight2", dtype=tf.float32, shape=[300, 10], initial_value=tf.random.normal(mean=0.0, stddev=0.01,shape=[300, 10], seed=42))
b2 = tf.Variable(name="Bias2", dtype=tf.float32, initial_value=tf.constant(0., shape=[10], dtype=tf.float32))

Calculate the output of the hidden layer

In [6]:
# Calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.sigmoid(hidden_out)
# Now calculate the hidden layer output - in this case, let's use a softmax activated
# output layer
out = tf.add(tf.matmul(hidden_out, W2), b2)
y_ = tf.nn.softmax(out, name="Softmax")

Generate cost and optimizer function

In [7]:
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = tf.reduce_mean(-tf.compat.v1.reduce_sum(y*tf.math.log(y_clipped),  reduction_indices=1))
# Add an optimizer
optimiser = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

Generate the assessment operation

In [8]:
# Finally setup the initialisation operator
init_op = tf.compat.v1.global_variables_initializer()

# Define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Start the session

In [9]:
# Start the session
with tf.compat.v1.Session() as sess:
  # Initialise the variables
  sess.run(init_op)
    
  total_batch = int(x_train.shape[0]/batch_size)
  print(total_batch)

  # Training cycle 
  for i in range(epochs):
    avg_cost = 0.
    # Loop over all batches
    for j in range(total_batch):
      x_batch=x_train[j*batch_size:(j+1)*batch_size]
      y_batch=y_train[j*batch_size:(j+1)*batch_size]
      _, c = sess.run([optimiser, cross_entropy], feed_dict={x: x_batch, y: y_batch})
      # Compute average loss
      avg_cost += c / total_batch
      
    print ("Epoch:", '%02d' % (i+1), "cost=", "{}".format(avg_cost))

  print ("Optimization Finished!")
  # Test model
  # Calculate accuracy for all examples
  print ("Accuracy:", accuracy.eval({x: x_test, y: y_test}))  

600
Epoch: 01 cost= 1.7231359573205296
Epoch: 02 cost= 1.2085440780719123
Epoch: 03 cost= 1.2555728024244315
Epoch: 04 cost= 1.2178202414512638
Epoch: 05 cost= 1.1141365026434267
Epoch: 06 cost= 1.0620982194940265
Epoch: 07 cost= 1.1417118513584135
Epoch: 08 cost= 1.0659333325425782
Epoch: 09 cost= 1.1525953752795857
Epoch: 10 cost= 1.1258779074748346
Optimization Finished!
Accuracy: 0.6651
